In [1]:
import inspire_agrivolt
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os
from collections import defaultdict
import pvdeg

from dask.distributed import Client, LocalCluster

In [2]:
cluster = LocalCluster(
    n_workers=4,        # Number of processes (can set to number of CPU cores you want to use)
    threads_per_worker=1,  # One thread per worker for true multiprocessing
    processes=True,     # Ensure you're using multiprocessing, not multithreading
)

client = Client(cluster)

client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 503.15 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40617,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 503.15 GiB
Comm: tcp://127.0.0.1:35149,Total threads: 1
Dashboard: http://127.0.0.1:37569/status,Memory: 125.79 GiB
Nanny: tcp://127.0.0.1:41317,


In [3]:
states = [
    "alabama", "arizona", "arkansas", "california", "colorado", "connecticut",
    "delaware", "florida", "georgia", "idaho", "illinois", "indiana",
    "iowa", "kansas", "kentucky", "louisiana", "maine", "maryland",
    "massachusetts", "michigan", "minnesota", "mississippi", "missouri", "montana",
    "nebraska", "nevada", "new_hampshire", "new_jersey", "new_mexico", "new_york",
    "north_carolina", "north_dakota", "ohio", "oklahoma", "oregon", "pennsylvania",
    "rhode_island", "south_carolina", "south_dakota", "tennessee", "texas", "utah",
    "vermont", "virginia", "washington", "west_virginia", "wisconsin", "wyoming"
]

confs = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10"]

In [5]:
files = defaultdict(list)

outputs_dir = "/projects/inspire/PySAM-MAPS/test-all-states"

for conf in confs:
    for state in states:

        path = f"{outputs_dir}/{state}/{conf}/*.nc"
        netcdfs = glob.glob(path)

        if len(netcdfs) > 0: 
            files[conf].extend(netcdfs)
        

In [6]:
for conf in confs:

    inspire_agrivolt.merge_pysam_out_nc_to_zarr(
        files=files[conf],
        zarr_path=f"/projects/inspire/PySAM-MAPS/test-all-states/merged/{conf}.zarr",
        engine="netcdf4"
    )

/home/tford/.conda-envs/geospatial/lib/python3.9/site-packages/numpy/core/numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')
/home/tford/.conda-envs/geospatial/lib/python3.9/site-packages/numpy/core/numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')
/home/tford/.conda-envs/geospatial/lib/python3.9/site-packages/numpy/core/numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')
/home/tford/.conda-envs/geospatial/lib/python3.9/site-packages/numpy/core/numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')
/home/tford/.conda-envs/geospatial/lib/python3.9/site-packages/xarray/core/dataset.py:2553: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
 

In [ ]:
import dask.array as da

engine="netcdf4"
files=files["01"]
zarr_path="/projects/inspire/PySAM-MAPS/test-all-states/merged/01.zarr"

lat_set, lon_set = set(), set()
for file in files:
    with xr.open_dataset(file, engine=engine, chunks={}) as ds:
        lat_set.update(ds.latitude.values.tolist()) # add all elements in iterables to sets
        lon_set.update(ds.longitude.values.tolist())
        
unique_lat = np.sort(np.fromiter(lat_set, dtype=float))
unique_lon = np.sort(np.fromiter(lon_set, dtype=float))

example_ds = xr.open_dataset(files[0], engine=engine)

chunks = {
    "latitude": 40, 
    "longitude":10
}

sizes={
    "latitude":len(unique_lat), 
    "longitude":len(unique_lon), 
    "time":8760, 
    "distance":10
}

zarr_chunks = {
    "time":     sizes["time"],      # 8760
    "latitude": chunks["latitude"], # 40
    "longitude":chunks["longitude"],# 10
    "distance": sizes["distance"]  # 10
}    

data_vars = {}
for data_var in example_ds.data_vars:
    
    dims = example_ds[data_var].dims
    
    shape = tuple(sizes[d] for d in dims)
    chunk_shape = tuple(chunks.get(d, sizes[d]) for d in dims)
    dtype = example_ds[data_var].dtype

    data_vars[data_var] = (dims, da.full(shape, np.nan, dtype=dtype, chunks=chunk_shape))
    
template = xr.Dataset(
    coords={
        "latitude":unique_lat,
        "longitude":unique_lon,
        "time":example_ds.time,
        "distance":np.arange(10)
    },
    data_vars=data_vars
)

template.to_zarr(zarr_path, mode='w')

merged = xr.open_zarr(zarr_path, consolidated=False)

for file in files:
    ds = xr.open_dataset(file, chunks={}, engine="netcdf4") # dataset to insert into zarr store
    
    lat_inds = np.searchsorted(unique_lat, ds.latitude.values)
    lon_inds = np.searchsorted(unique_lon, ds.longitude.values)

    # zarr region to write
    region = { 
        "latitude": slice(lat_inds[0], lat_inds[-1] + 1),
        "longitude": slice(lon_inds[0], lon_inds[-1] + 1),
        "time":slice(0,8760),
        "distance":slice(0,10)
    }

    # original values from region being written to
    existing_block = merged.isel(
        latitude=region["latitude"], 
        longitude=region["longitude"]
    )

    patched = existing_block.combine_first(ds)

    # realign dask chunks with zarr chunks
    chunk_map = {dim: zarr_chunks[dim] for dim in patched.dims}

    patched = patched.chunk(chunk_map)
    
    patched.to_zarr(
        zarr_path, 
        region=region, 
        mode="r+"
    )
    
    #ref_var = "annual_poa"
    #mask = np.isnan(existing_block[ref_var]) & ~np.isnan(ds[ref_var])
    #mask = ( existing_block[ref_var].isnull() & ds[ref_var].notnull() )

    #patched = xr.where(mask, ds, existing_block) 
    
    # patch_data = {}
    # for var in ds.data_vars:
    #     patch_data[var] = (
    #         ds[var].dims, 
    #         np.where(mask, ds[var].values, existing_block[var].values)
    #     )
                    
    # patch_coords = {
    #     "time": merged.time.values,
    #     "distance": merged.distance.values,
    #     "latitude": merged.latitude.values[region["latitude"]],
    #     "longitude": merged.longitude.values[region["longitude"]],
    # }
                    
    # patch = xr.Dataset(
    #     data_vars=patch_data,
    #     coords=patch_coords
    # )

In [ ]:
import pvdeg

pvdeg.geospatial.plot_sparse_analysis(
    xr.open_zarr(zarr_path),
    'tilt'
)

In [ ]:
existing_block.albedo.mean('time').isnull().compute()